### DataSet & DataLoader 살펴보기
- pytorch에서 배치크기만큼 데이터를 조절하기 위한 메커니즘
- Dataset : 사용자 데이터를 기반으로 사용자 정의 클래스 작성
- DataLoad : 지정된 Dataset에서 지정된 배치 크기만큼 피처와 타깃을 추출하여 전달

In [1]:
# 0. Load Module
import torch
import torch.nn as nn
from torch.utils.data import Dataset, irisDFLoader

import numpy as np
import pandas as pd

In [5]:
# 1. Load Data
x_data = torch.IntTensor(
    [[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60], [50, 60, 70]]
)
y_data = torch.FloatTensor([[20], [30], [40], [50], [60]])

print(x_data.shape, x_data.ndim, y_data.shape, y_data.ndim)

torch.Size([5, 3]) 2 torch.Size([5, 1]) 2


In [10]:
# 2. Create DataSet
# 1) TensoririsDFset 활용 : Dataset의 sub_class
from torch.utils.data import TensorDataset

dataset = TensorDataset(x_data, y_data)
dataset.tensors

# 주의 : x, y data의 행 번호가 맞아야 실행된다!

(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20.],
         [30.],
         [40.],
         [50.],
         [60.]]))

In [9]:
# __getitem__() 메서드 호출
dataset[0]

(tensor([10, 20, 30], dtype=torch.int32), tensor([20.]))

In [20]:
# 2) 사용자 정의 데이터셋 생성
# (1) Load file
irisDF = pd.read_csv('iris.csv')
irisDF.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [21]:
# (2) feature : numpy로 가져오기
irisNP = np.loadtxt('iris.csv', delimiter=',', usecols=[0, 1, 2, 3], skiprows=1)
irisNP.shape

(150, 4)

In [22]:
# (3) 사용자 정의 Dataset class
# - callback function
class IrisDataset(Dataset):
    def __init__(self, x_data, y_data):  # 초기화 함수
        super.__init__()
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data  # x_data가 DataFrame이면 values를 반환
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.FloatTensor(y_data)

    def __len__(self):  # 갯수 확인 함수
        return self.target.shape[0]

    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [26]:
# check datatype
print(
    type(irisDF), 
    type(irisNP),
    irisDF.__class__.__name__,
    irisNP.__class__.__name__,
    sep="\n"
)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
DataFrame
ndarray
